In [1]:
from itertools import count
import deepcut
from deep_translator import GoogleTranslator
from difflib import get_close_matches
import json
import re
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import pyodbc
import csv

from nltk.tokenize import sent_tokenize, word_tokenize
from pythainlp import sent_tokenize, word_tokenize, correct, spell, Tokenizer
import gensim
from gensim.models import Word2Vec

from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

import numpy as np
from pythainlp.util import dict_trie, Trie 
from matplotlib.colors import ListedColormap

In [37]:
connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')  

cursor = connect_db.cursor()

SafetyAudit = pd.read_sql("SELECT * FROM [FindingDetails];", connect_db)
    
Custom_Dict = pd.read_csv('../SMIT_Data/DataForModel/Raw_Dictionary.csv',encoding='utf-8')
DictCorrect = Custom_Dict['correct'].tolist()

DictCorrect = list(set(DictCorrect))

modelPath = "../Model/SentenceTransformer"
model = SentenceTransformer(modelPath)

SafetyAuditDetails = SafetyAudit['Details'].tolist()
SafetyAuditCleansingDetails = SafetyAudit['CleansingDetails'].tolist()
SafetyAuditTranslateDetails = SafetyAudit['TransCleansingDetails'].tolist()
SafetyAuditTopic = SafetyAudit['Topic'].tolist()
SafetyAuditTof = SafetyAudit['TypeOfFinding'].tolist()

trie = Trie(DictCorrect)

In [ ]:
tbFinding = pd.read_csv('./SMIT_Data/Classification_tbFinding/tbFinding.csv', encoding='utf-8')

Finding = tbFinding['Finding'].tolist()
Translate_Finding = []


for sentence in Finding:
    Translate_Finding.append(GoogleTranslator(source = 'auto', target = 'en').translate(sentence))

Create Dict for 5 Clusters : Type of Finding

In [ ]:
# ------------------ Prepared Cluster : Unsafe Action ------------------

ENData_UnsafeAction = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Unsafe Action", "TransCleansingDetails"]).tolist()
THData_UnsafeAction = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Unsafe Action", "CleansingDetails"]).tolist()
Data_UnsafeAction = ' '.join(ENData_UnsafeAction+THData_UnsafeAction)

CountVector = CountVectorizer(stop_words='english') 
doc_array = CountVector.fit_transform([Data_UnsafeAction]).toarray() 

# ListWords_UnsafeAction = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_UnsafeAction = list(set(word_tokenize(Data_UnsafeAction, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Unsafe Condition ------------------

ENData_UnsafeCondition = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Unsafe Condition", "TransCleansingDetails"]).tolist()
THData_UnsafeCondition = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Unsafe Condition", "CleansingDetails"]).tolist()
Data_UnsafeCondition = ' '.join(ENData_UnsafeCondition+THData_UnsafeCondition)

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_UnsafeCondition]).toarray() 

# ListWords_UnsafeCondition = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_UnsafeCondition = list(set(word_tokenize(Data_UnsafeCondition, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Near Miss ------------------

ENData_NearMiss = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Near Miss", "TransCleansingDetails"]).tolist()
THData_NearMiss = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Near Miss", "CleansingDetails"]).tolist()
Data_NearMiss = ' '.join(ENData_NearMiss+THData_NearMiss)

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_NearMiss]).toarray() 

# ListWords_NearMiss = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_NearMiss = list(set(word_tokenize(Data_NearMiss, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : HNM ------------------

ENData_HNM = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "HNM", "TransCleansingDetails"]).tolist()
THData_HNM = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "HNM", "CleansingDetails"]).tolist()
Data_HNM = ' '.join(ENData_HNM+THData_HNM)

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_HNM]).toarray() 

# ListWords_HNM = list(set(list(CountVector.get_feature_names_out()))) 

ListWords_HNM = list(set(word_tokenize(Data_HNM, custom_dict=trie, engine='newmm')))

# ------------------ Prepared Cluster : Accident ------------------
ENData_Accident = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Accident", "TransCleansingDetails"]).tolist() 
THData_Accident = (SafetyAudit.loc[SafetyAudit["Type of finding"] == "Accident", "CleansingDetails"]).tolist() 
Data_Accident = ' '.join(ENData_Accident+THData_Accident) 

CountVector = CountVectorizer(stop_words='english', vocabulary=DictCorrect) 
doc_array = CountVector.fit_transform([Data_Accident]).toarray() 
 
# ListWords_Accident = list(set(list(CountVector.get_feature_names_out())))   

ListWords_Accident = list(set(word_tokenize(Data_Accident, custom_dict=trie, engine='newmm')))

Create Dict for 5 Clusters : Topic

In [ ]:
# All_Encode_CountVector = model.encode(CountVector.get_feature_names_out()) 

# doc_array = doc_array[0]

# Temp_Encode_CountVector = []

# for i in range(len(All_Encode_CountVector)):
#     temp = []
#     temp.append(sum(All_Encode_CountVector[i]))
#     temp.append(doc_array[i])

#     Temp_Encode_CountVector.append(temp)
    
# Encode_CountVector = np.array(Temp_Encode_CountVector)

# print(All_Encode_CountVector)
# y = [1] * 1000
# y.extend([2] * 299)

# cmap_bold = ListedColormap(['#FF0000'])

# clf = NearestCentroid()
# clf.fit(Encode_CountVector, y)
# y_pred = clf.predict(Encode_CountVector)

# x_min, x_max = Encode_CountVector[:, 0].min() - 1, Encode_CountVector[:, 0].max() + 1
# y_min, y_max = Encode_CountVector[:, 1].min() - 1, Encode_CountVector[:, 1].max() + 1

# xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.2), np.arange(y_min, y_max, 0.2))

# Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Z = Z.reshape(xx.shape)
# plt.figure()

# plt.scatter(Encode_CountVector[:, 0], Encode_CountVector[:, 1], c=doc_array, cmap=cmap_bold, edgecolor='k', s=20)
# plt.title("Cluster Unsafe Action")

# plt.show()

Show Cluster in Graph

In [ ]:
# from statistics import mean

# # Mean_Cluster = mean(list(doc_array))

# Cluster_UnsafeAction = [] 
# Cluster_UnsafeCondition = []
# Cluster_NearMiss = []
# Cluster_HNM = []
# Cluster_Accident = []

# for i in range(len(ListWords_UnsafeAction)):
#     temp = []
#     temp.append(ListWords_UnsafeAction[i])
#     temp.append(doc_array[i])
#     Cluster_UnsafeAction.append(temp)

# for i in range(len(ListWords_UnsafeCondition)):    
#     temp = []
#     temp.append(ListWords_UnsafeCondition[i])
#     temp.append(doc_array[i])
#     Cluster_UnsafeCondition.append(temp)

# for i in range(len(ListWords_NearMiss)):
#     temp.append(ListWords_NearMiss[i])
#     temp.append(doc_array[i])
#     Cluster_NearMiss.append(temp)

# for i in range(len(ListWords_HNM)):
#     temp.append(ListWords_HNM[i])
#     temp.append(doc_array[i])
#     Cluster_HNM.append(temp)

# for i in range(len(ListWords_Accident)):
#     temp.append(ListWords_Accident[i])
#     temp.append(doc_array[i])
#     Cluster_Accident.append(temp)    
    
# Cluster_UnsafeAction = pd.DataFrame(data = Cluster_UnsafeAction, columns = ['words', 'frequency'])
# Cluster_UnsafeCondition = pd.DataFrame(data = Cluster_UnsafeCondition, columns = ['words', 'frequency'])
# Cluster_NearMiss = pd.DataFrame(data = Cluster_NearMiss, columns = ['words', 'frequency'])
# Cluster_HNM = pd.DataFrame(data = Cluster_HNM, columns = ['words', 'frequency'])
# Cluster_Accident = pd.DataFrame(data = Cluster_Accident, columns = ['words', 'frequency'])

In [ ]:
## Get TF-IDF Value of Every Cluster

# docs = [ListWords_UnsafeAction, ListWords_UnsafeCondition, ListWords_NearMiss, ListWords_HNM, ListWords_Accident]

# vocab = set(ListWords_UnsafeAction + ListWords_UnsafeCondition + ListWords_NearMiss + ListWords_HNM + ListWords_Accident)

# def tfidf(word, sentence):
#     tf = sentence.count(word) / len(sentence)
#     idf = np.log10(len(docs) / sum([1 for doc in docs if word in doc]))
#     return round(tf*idf, 4)

# TFIDF_UnsafeAction = []
# TFIDF_UnsafeCondition = []
# TFIDF_NearMiss = []
# TFIDF_Hnm = []
# TFIDF_Accident = []

# for word in vocab:
#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_UnsafeAction))
#     TFIDF_UnsafeAction.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_UnsafeCondition))
#     TFIDF_UnsafeCondition.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_NearMiss))
#     TFIDF_NearMiss.append(temp)

#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_HNM))
#     TFIDF_Hnm.append(temp)
    
#     temp = []
#     temp.append(word)
#     temp.append(tfidf(word, ListWords_Accident))
#     TFIDF_Accident.append(temp)

In [38]:
def CreateFrequency(word):
    Value = 6
    if word in ListWords_UnsafeAction:
        Value -= 1
    if word in ListWords_UnsafeCondition:
        Value -= 1
    if word in ListWords_NearMiss:
        Value -= 1
    if word in ListWords_HNM:
        Value -= 1
    if word in ListWords_Accident:
        Value -= 1
    if Value == 6:
        return 0
    else:
        return Value

Cluster_UnsafeAction = []
Cluster_UnsafeCondition = []
Cluster_NearMiss = []
Cluster_HNM = []
Cluster_Accident = []

for word in ListWords_UnsafeAction:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_UnsafeAction.append(temp)

for word in ListWords_UnsafeCondition:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_UnsafeCondition.append(temp)

for word in ListWords_NearMiss:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_NearMiss.append(temp)

for word in ListWords_HNM:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_HNM.append(temp)

for word in ListWords_Accident:
    temp = []
    temp.append(word)
    temp.append(CreateFrequency(word))
    Cluster_Accident.append(temp)

1. เปลี่ยนรูปแบบการดึงข้อมูล tbFinding ให้ดึงจาก Database 
2. เพิ่มการแปลภาษา tbFinding ก่อนดึงข้อมูลมาจาก Database

In [39]:
import pyodbc
import csv

ClassifyTopic = []

tbFinding = pd.read_csv('../SMIT_Data/Classification_tbFinding/tbFindingResult.csv', encoding='utf-8')

tbFindingNo = []
tbFindingArea = tbFinding['Area'].tolist()
tbFindingSubArea = []
tbFindingContractor = []
tbFindingTof = []
tbFindingTopic = []
tbFindingFinding = tbFinding['Finding'].tolist()
tbFindingTranslate = tbFinding['TranslateFinding'].tolist()

Custom_Dict = pd.read_csv('../SMIT_Data/DataForModel/Raw_Dictionary.csv',encoding='utf-8')
DictCorrect = Custom_Dict['correct'].tolist()

trie = Trie(DictCorrect)

for index in range(len(tbFindingTranslate)):
    Data = word_tokenize(tbFindingTranslate[index], custom_dict=trie, engine='newmm')
    Finding = "-"
    temp = []
    tempmax = 0
    max = -1
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_UnsafeAction])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_UnsafeAction]):
                tempmax += Cluster_UnsafeAction[ListWords_UnsafeAction.index(i)][1]
        if tempmax > max:
            max = tempmax
            Finding = "Unsafe Action"

    tempmax = 0
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_UnsafeCondition])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_UnsafeCondition]):
                tempmax += Cluster_UnsafeCondition[ListWords_UnsafeCondition.index(i)][1]
        if tempmax > max:
            max = tempmax
            Finding = "Unsafe Condition"

    tempmax = 0
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_NearMiss])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_NearMiss]):
                tempmax += Cluster_NearMiss[ListWords_NearMiss.index(i)][1]
        if tempmax > max:
            max = tempmax
            Finding = "Near Miss"

    tempmax = 0
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_HNM])) > 3:
        for i in set([x.lower() for x in Data]):
            if i in [j[0] for j in Cluster_HNM]:
                tempmax += Cluster_HNM[ListWords_HNM.index(i)][1]
        if tempmax > max:
            max = tempmax
            Finding = "HNM"

    tempmax = 0
    if len(set([x.lower() for x in Data]) & set([i[0] for i in Cluster_Accident])) > 3:
        for i in (set([x.lower() for x in Data])):
            if i in ([j[0] for j in Cluster_Accident]):
                tempmax += Cluster_Accident[ListWords_Accident.index(i)][1]
        if tempmax > max:
            max = tempmax
            Finding = "Accident"
    
    tbFindingNo.append(str(index+1)) 
    tbFindingSubArea.append('Unknown')  
    tbFindingContractor.append('Unknown')     
    tbFindingTof.append(Finding)
    tbFindingTopic.append('Unknown')

    temp.append(str(index+1)) 
    temp.append(tbFindingArea[index])
    temp.append('Unknown')
    temp.append('Unknown') 
    temp.append(Finding)
    temp.append(tbFindingFinding[index]) 

    ClassifyTopic.append(temp)

connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "TONY",
                            Database = "SMIT3",
                            uid = 'Local_SMIT3.0',
                            pwd = 'Tony123456',
                            Trusted_Connection = 'yes')      

ResponseClassification_TbFinding = pd.read_sql("SELECT * FROM Classification_TbFinding ORDER BY FindingNo ASC;", connect_db)

ResponseClassification_TbFinding = list(ResponseClassification_TbFinding.itertuples(index=False, name=None))

if(len(ResponseClassification_TbFinding) > 0):
    cursor = connect_db.cursor()
    Query = "DELETE FROM [Classification_TbFinding];"
    cursor.execute(Query)
    ResponseClassification_TbFinding = []

Classification_TbFinding = list(zip(tbFindingNo, tbFindingArea, tbFindingSubArea, 
                                    tbFindingContractor, tbFindingTof, tbFindingTopic, 
                                    tbFindingFinding, tbFindingTranslate))
                            
# print(len(Prepared_Safety_Audit), len(ResponsePreparedFindingDetail))

# Prepared_Safety_Audit = list(set(Prepared_Safety_Audit) - set(ResponsePreparedFindingDetail))

cursor = connect_db.cursor()
Query = "INSERT INTO [Classification_TbFinding] VALUES (?, ?, ?, ?, ?, ?, ?, ?);"

cursor.executemany(Query, Classification_TbFinding)
connect_db.commit()

Head = ['FindingNo', 'Area', 'SubArea', 'Contractor', 'TypeOfFinding', 'Topic', 'Details', 'TranslateDetails']

with open('../SMIT_Data/Classification_TbFinding.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f)
  write.writerow(Head)
  write.writerows(Classification_TbFinding)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
